In [45]:
import pymongo
import pandas as pd

In [3]:
user = 'Estudante'
pwd = 'Estudante20202'
connection_string = 'mongodb+srv://{}:{}@bdrnr.mw4lf.mongodb.net/bdrnr_db?retryWrites=true&w=majority'.format(user, pwd)
client = pymongo.MongoClient(connection_string)
db = client.bdrnr_db

In [46]:
resultado = db.G01.find()
df = pd.DataFrame(list(resultado))
df.head()

,_id,MES_ANO_ACIDENTE,DATA_ACIDENTE,TIPO_DO_ACIDENTE,UF_MUNIC_ACIDENTE,AGENTE_CAUSADOR_ACIDENTE,ORIGEM_DE_CADASTRAMENTO_CAT,EMITENTE_CAT,DATA_EMISSAO_CAT,ESPECIE_DO_BENEFICIO,...,DESC_OCUPACAO,CID_10,SUBCATEGORIA,CAT_CID_10,CATEGORIA,CNAE_EMP,CNAE_DENOMINACAO,COD_MUNIC,MUNICIPIO,POPULACAO
0,5fad600a382b8ee116082397,2019-11-01,2019-11-01,Tipico,Tocantins,Metal - Inclui Liga,Internet,Empregador,2019-12-10,Pa,...,Trabalhador de servicos de limpeza e conservac...,C448,Neoplasia maligna da pele com lesao invasiva,C44,Outras neoplasias malignas da pele,8121,Limpeza em predios e em domicilios,430460,Canoas,344957
1,5fad600a382b8ee116082398,2020-01-01,2020-01-09,Tipico,NaN,"Politriz, Lixadora,",Internet,Empregador,2020-01-10,Pa,...,Soldador,C765,Neoplasia maligna do membro inferior,C76,Neoplasia maligna de outras localizacoes e de ...,5250,Atividades relacionadas a organizacao do trans...,431560,Rio Grande,210005
2,5fad600a382b8ee116082399,2019-12-01,2019-12-11,Tipico,NaN,Ferramenta Portatil,Internet,Empregador,2019-12-12,Pa,...,Tecnico florestal,D092,Carcinoma in situ do olho,D09,Carcinoma in situ de outras localizacoes e das...,210,Producao florestal - florestas plantadas,430190,Barra do Ribeiro,13423
3,5fad600a382b8ee11608239a,2019-09-01,2019-09-25,Tipico,NaN,Chao - Superficie Ut,Internet,Empregador,2019-10-01,Pa,...,Inspetor de servicos de transportes rodoviario...,E639,Deficiencia nutricional nao especificada,E63,Outras deficiencias nutricionais,7112,Servicos de engenharia,431490,Porto Alegre,1479101
4,5fad600a382b8ee11608239b,2020-02-01,2020-02-28,Tipico,NaN,Gas Carbonico (Dioxi,Internet,Empregador,2020-03-02,Pa,...,Auxiliar de enfermagem,G430,Enxaqueca sem aura [enxaqueca comum],G43,Enxaqueca,8610,Atividades de atendimento hospitalar,431490,Porto Alegre,1479101


1- Para nascidos neste século (DATA_NASCIMENTO > "2001"), listar AGENTE_CAUSADOR_ACIDENTE, DATA_NASCIMENTO e SEXO, ordenados por DATA_NASCIMENTO descendente, SEXO e AGENTE_CAUSADOR_ACIDENTE.

In [90]:
resultado = db.G01.find( { 'DATA_NASCIMENTO': { '$gt': '2001' } },
                         { 'AGENTE_CAUSADOR_ACIDENTE':1,
                           'DATA_NASCIMENTO':1,
                           'SEXO':1,
                           '_id':0 }
                       ).sort([('DATA_NASCIMENTO', -1),
                               ('SEXO', 1),
                               ('AGENTE_CAUSADOR_ACIDENTE', 1)])

df = pd.DataFrame(list(resultado))
df.head(10)

,AGENTE_CAUSADOR_ACIDENTE,SEXO,DATA_NASCIMENTO
0,"Veiculo, Nic",Masculino,2004-07-14
1,Queda de Pes. em Mes,Feminino,2003-01-27
2,Empilhadeira,Masculino,2002-04-13
3,"Caixa, Engradado, Ca",Masculino,2002-01-17
4,"Tesoura, Tesourao- F",Masculino,2002-01-03
5,Superficie e Estrutu,Masculino,2001-09-11
6,Metal - Inclui Liga,Masculino,2001-09-09
7,Escada Permanente Cu,Feminino,2001-07-28
8,Escada Permanente Cu,Feminino,2001-06-29
9,"Faca, Facao- Ferrame",Masculino,2001-05-27


2- Ordenados por número de acidentes em ordem decrescente, listar MUNICIPIO, POPULACAO e número de acidentes, dos 20 municípios com mais acidentes.

In [88]:
resultado = db.G01.aggregate([
                        {"$group":{"_id": {"MUNICIPIO":"$MUNICIPIO", 
                                           "POPULACAO":"$POPULACAO"},
                                   "NUMACIDS": {"$sum":1}}},
                        {"$sort":{"NUMACIDS" : -1}},
                        {"$limit":20} ])

print("{:30}\t{:20}\t{:23}".format("Município", "POPULACAO", "Número acidentes" ))
print("{:_<30}\t{:_<20}\t{:_<23}".format("_", "_", "_"))
for documento in resultado:    
    print("{:30}\t{:20}\t{:23}".format(documento["_id"]["MUNICIPIO"],
            documento["_id"]["POPULACAO"],
            documento["NUMACIDS"]))

Município                     	POPULACAO           	Número acidentes       
______________________________	____________________	_______________________
Porto Alegre                  	             1479101	                    297
Caxias do Sul                 	              504069	                    107
Passo Fundo                   	              201767	                     63
Canoas                        	              344957	                     43
Santa Maria                   	              280505	                     40
Erechim                       	              105059	                     35
Lajeado                       	               82951	                     34
Sao Leopoldo                  	              234947	                     31
Gravatai                      	              279398	                     29
Santa Cruz do Sul             	              129427	                     26
Marau                         	               43435	                     22
Novo Hamburg

3- Ordenados em ordem decrescente por número de acidentes, listar as 15 primeiras denominações das atividades econômicas (CNAE_DENOMINACAO), municípios e número de acidentes. (dica, agrupar por CNAE_DENOMINACAO e MUNICIPIO)

In [126]:
resultado = db.G01.aggregate([
                        {"$group":{"_id": {"CNAE_DENOMINACAO":"$CNAE_DENOMINACAO", 
                                           "MUNICIPIO":"$MUNICIPIO"},
                                   "NUMACIDS": {"$sum":1}}},
                        {"$sort":{"NUMACIDS" : -1}},
                        {"$limit":15} ])

print("{:30}\t{:20}\t{:23}".format("CNAE_DENOMINACAO", "Município", "Número acidentes" ))
print("{:_<30}\t{:_<20}\t{:_<23}".format("_", "_", "_"))
for documento in resultado:    
    print("{:30}\t{:20}\t{:23}".format(documento["_id"]["CNAE_DENOMINACAO"],
            documento["_id"]["MUNICIPIO"],
            documento["NUMACIDS"]))

CNAE_DENOMINACAO              	Município           	Número acidentes       
______________________________	____________________	_______________________
Atividades de atendimento hospitalar	Porto Alegre        	                    132
Atividades de atendimento hospitalar	Passo Fundo         	                     22
Abate de suinos, aves e outros pequenos animais	Lajeado             	                     18
Atividades de atendimento hospitalar	Caxias do Sul       	                     16
Atividades de Correio         	Porto Alegre        	                      9
Atividades de atencao a saude humana nao especificadas anteriormente	Ijui                	                      9
Fabricacao de cabines, carrocerias e reboques para veiculos automotores	Caxias do Sul       	                      9
Comercio varejista de mercadorias em geral, com predominancia de produtos alimenticios - hipermercados e supermercados	Porto Alegre        	                      8
Abate de suinos, aves e outros pequeno

4- Ordenados por mês/ano, denominações das atividades econômicas e número de acidentes, filtrando por agente causador de acidente, sexo e atividade econômica. (escolham uma trinca (agente causador de acidente, sexo e atividade econômica) que permita a consulta retornar entre 5 e 30 linhas).

5- Listar os 10 municípios e respectivas populações com as maiores taxas de acidentes por 100.000 habitantes.

In [132]:
resultado = db.G01.aggregate([
                        {"$group":{"_id": {"MUNICIPIO":"$MUNICIPIO", 
                                           "POPULACAO":"$POPULACAO"},
                                  "NUMACIDS": {"$sum":1 / '$toDouble(POPULACAO)'}}},
                        {"$sort":{"NUMACIDS" : -1}},
                        {"$limit":10} ])

print("{:30}\t{:20}\t{:23}".format("Município", "POPULACAO", "Número acidentes" ))
print("{:_<30}\t{:_<20}\t{:_<23}".format("_", "_", "_"))
for documento in resultado:    
    print("{:30}\t{:20}\t{:23}".format(documento["_id"]["MUNICIPIO"],
            documento["_id"]["POPULACAO"],
            documento["NUMACIDS"]))

TypeError: unsupported operand type(s) for /: 'int' and 'str'